<a href="https://colab.research.google.com/github/statisticianinstilettos/ClaireBot/blob/main/Monitoring_LLM_Systems_in_Production_with_Comet_Opik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LLM Observability and Monitoring Tutorial with my Recipe Generator App

This example personal project is designed to practice establishing proper montioring best practices for a live LLM app. In this tutorial, I'll walk you through how to build a very simple yet powerful LLM app, and show how to design the monitoring metrics and dashboards to facilitate detecting and root causing issues that might arise in production.


## Preparing our environment

First, we will install the required libraries and set up our OpenAI API keys. Check out Opik config parameters here: https://www.comet.com/docs/opik/tracing/sdk_configuration

In [ ]:
%pip install -U opik openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.3/371.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.1 MB/s eta 0:00:00


In [ ]:
import opik

opik.configure(use_local=False)

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "statisticianinstilettos" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


In [ ]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [ ]:
from opik import track, opik_context
from opik.integrations.openai import track_openai
from openai import OpenAI

os.environ["OPIK_PROJECT_NAME"] = "recipe-generator"

client = OpenAI()
openai_client = track_openai(client)

# Building a simple recipe generator

I'm using the OpenAI API to generate a response, and Opik to track and evaluate it.

In [ ]:
#This function performs the API call to openAI to request the recipe based on the prompt. Simply enter the dish name!
#Because I'm using a function, I'm using the Opik @track decorator to log my traces.

@opik.track(flush=True)
def generate_recipe(dish_name):

    prompt = f"""
    You are a world-class home chef and recipe developer. Generate a step-by-step recipe for {dish_name}.
    Include a list of ingredients with quantities, clear cooking instructions, preparation time, cooking time, and serving size.
    If applicable, provide useful tips, optional variations, or suggestions for substitutes.
    Keep the recipe straightforward and easy to follow for a novice at home cook.
    """

    # Request recipe generation from OpenAI's completions model
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a world-class chef and recipe developer."},  # system message
            {"role": "user", "content": prompt}  # User's input prompt
        ],
        max_tokens=500,
        temperature=0.7,
        n=1
    )

    # Add user feedback score to the trace in Opik
    opik_context.update_current_trace(
    feedback_scores=[
        {"name": "user_feedback", "value": 1.0, "reason": "It looks good."}
        ]
    )

    return response



In [ ]:
# Example usage
dish_name = "red velet cupcakes"
recipe = generate_recipe(dish_name)

#extract just the recipe text and print
print(recipe.choices[0].to_dict()['message']['content'])

OPIK: Started logging traces to the "recipe-generator" project at https://www.comet.com/opik/statisticianinstilettos/redirect/projects?name=recipe-generator.


**Cheddar Dill Biscuits**

**Ingredients:**
- 4 cups all-purpose flour
- 1 packet instant yeast
- 3 large eggs
- 1 cup milk
- 1 cup melted butter
- 1 cup shredded cheddar cheese
- 2 tablespoons dried dill
- 1 teaspoon salt

**Instructions:**

**Preparation Time:** 20 minutes
**Cooking Time:** 15-20 minutes
**Serving Size:** 12 biscuits

1. Preheat your oven to 375°F (190°C) and line a baking sheet with parchment paper.
   
2. In a large mixing bowl, combine the flour, instant yeast, shredded cheddar cheese, dried dill, and salt. Mix well to combine all the dry ingredients.

3. In a separate bowl, whisk together the eggs, milk, and melted butter until well combined.

4. Make a well in the center of the dry ingredients and pour the wet mixture into it. Use a spatula to gently fold the wet and dry ingredients together until a soft dough forms. Do not overmix.

5. Turn the dough out onto a lightly floured surface and gently knead it a few times until it comes together.

6. Roll out the dou